In [1]:
import cmlapi
from datetime import datetime, timedelta
import json
import inspect

In [2]:


print("=== CML API Methods Comparison ===")
print("Comparing getTimeSeries vs listUsage methods")

# Initialize client
client = cmlapi.default_client()

print("\n" + "="*60)
print("1. ANALYZING getTimeSeries METHOD")
print("="*60)

# Inspect getTimeSeries method
if hasattr(client, 'get_time_series'):
    print("✅ get_time_series method found")
    
    try:
        sig = inspect.signature(client.get_time_series)
        print(f"Signature: get_time_series{sig}")
        
        doc = client.get_time_series.__doc__
        if doc:
            print(f"Documentation preview: {doc[:300]}...")
        
    except Exception as e:
        print(f"Error inspecting get_time_series: {e}")
else:
    print("❌ get_time_series method not found")

# Test getTimeSeries with sample data
print("\n--- Testing getTimeSeries Response Structure ---")
try:
    end_time = datetime.now()
    start_time = end_time - timedelta(days=7)
    
    time_range = {
        "created_time": {
            "min": start_time.strftime("%Y-%m-%d %H:%M:%S"),
            "max": end_time.strftime("%Y-%m-%d %H:%M:%S")
        }
    }
    
    result = client.get_time_series(
        series_type='cpu',
        time_range_search_filter=json.dumps(time_range)
    )
    
    result_dict = result.to_dict()
    
    print("✅ getTimeSeries successful!")
    print("Response structure:")
    print(f"  - series_type: {result_dict.get('series_type')}")
    print(f"  - result keys: {list(result_dict.get('result', {}).keys())}")
    
    if 'values' in result_dict.get('result', {}):
        values = result_dict['result']['values']
        if values:
            print(f"  - Data points: {len(values)}")
            print(f"  - Sample data point: {values[0]}")
            print(f"  - Available fields in data points: {list(values[0].keys())}")
        else:
            print("  - No data points returned")
    
    print("\n📊 getTimeSeries provides:")
    print("  ✅ Time series data (timestamps + counts)")
    print("  ✅ Resource type (cpu/memory/gpu)")
    print("  ❌ NO user information")
    print("  ❌ NO workload details")
    print("  ❌ NO project information")
    print("  ❌ NO job names or identifiers")
    
except Exception as e:
    print(f"❌ getTimeSeries test failed: {e}")

print("\n" + "="*60)
print("2. ANALYZING listUsage METHOD")
print("="*60)

# Check for listUsage method
usage_methods = [attr for attr in dir(client) if 'usage' in attr.lower()]
print(f"Found usage-related methods: {usage_methods}")

list_usage_methods = [attr for attr in dir(client) if 'list' in attr.lower() and 'usage' in attr.lower()]
print(f"Found list_usage methods: {list_usage_methods}")

# Try different variations of the method name
method_variations = [
    'list_usage',
    'listUsage', 
    'list_usages',
    'get_usage',
    'get_usages',
    'usage_list'
]

found_method = None
for method_name in method_variations:
    if hasattr(client, method_name):
        found_method = method_name
        print(f"✅ Found method: {method_name}")
        break

if found_method:
    method_func = getattr(client, found_method)
    
    # Inspect the method
    try:
        sig = inspect.signature(method_func)
        print(f"Signature: {found_method}{sig}")
        
        doc = method_func.__doc__
        if doc:
            print(f"Documentation: {doc[:500]}...")
    except Exception as e:
        print(f"Error inspecting {found_method}: {e}")
    
    # Test the method
    print(f"\n--- Testing {found_method} Response Structure ---")
    try:
        # Try calling with time range (common pattern)
        end_time = datetime.now()
        start_time = end_time - timedelta(days=7)
        
        time_range = {
            "created_time": {
                "min": start_time.strftime("%Y-%m-%d %H:%M:%S"),
                "max": end_time.strftime("%Y-%m-%d %H:%M:%S")
            }
        }
        
        # Try different parameter combinations
        param_combinations = [
            {},  # No parameters
            {'page_size': 10},  # With page size
            {'time_range_search_filter': json.dumps(time_range)},  # With time range
            {'search_filter': '{}'},  # Empty search filter
        ]
        
        for i, params in enumerate(param_combinations):
            try:
                print(f"  Attempt {i+1}: {found_method}({params})")
                result = method_func(**params)
                
                print(f"    ✅ SUCCESS! Result type: {type(result)}")
                
                # Convert to dict if possible
                if hasattr(result, 'to_dict'):
                    result_dict = result.to_dict()
                    print(f"    Result keys: {list(result_dict.keys())}")
                    
                    # Look for usage data
                    for key, value in result_dict.items():
                        if isinstance(value, list) and value:
                            print(f"    - {key}: list with {len(value)} items")
                            first_item = value[0]
                            if isinstance(first_item, dict):
                                print(f"      First item fields: {list(first_item.keys())}")
                                print(f"      Sample data: {first_item}")
                            break
                    break
                        
            except Exception as e:
                print(f"    ❌ Failed: {e}")
                continue
                
    except Exception as e:
        print(f"❌ {found_method} test failed: {e}")
else:
    print("❌ No list_usage method found")
    
    # Search more broadly for usage-related methods
    print("\n--- Searching for all usage-related methods ---")
    all_methods = [attr for attr in dir(client) if not attr.startswith('_')]
    usage_related = [method for method in all_methods if any(keyword in method.lower() 
                     for keyword in ['usage', 'utilization', 'consumption', 'resource', 'workload'])]
    
    if usage_related:
        print("Found potentially relevant methods:")
        for method in usage_related:
            print(f"  - {method}")
            
        # Try the most promising ones
        promising_methods = [m for m in usage_related if 'list' in m.lower() or 'get' in m.lower()][:3]
        
        for method_name in promising_methods:
            print(f"\n--- Testing {method_name} ---")
            try:
                method_func = getattr(client, method_name)
                sig = inspect.signature(method_func)
                print(f"  Signature: {method_name}{sig}")
                
                # Try calling with minimal parameters
                result = method_func()
                print(f"  ✅ {method_name} callable! Type: {type(result)}")
                
                if hasattr(result, 'to_dict'):
                    result_dict = result.to_dict()
                    print(f"  Keys: {list(result_dict.keys())}")
                    
            except Exception as e:
                print(f"  ❌ {method_name} failed: {e}")
    else:
        print("No usage-related methods found")

print("\n" + "="*60)
print("3. METHOD COMPARISON SUMMARY")
print("="*60)

print("""
📊 getTimeSeries Analysis:
  ✅ Provides: Time series data with timestamps and resource counts
  ✅ Good for: Resource utilization trends over time
  ❌ Limited: No user, project, or workload context
  ❌ Missing: Job names, creators, duration, status
  
🎯 Ideal for: Infrastructure monitoring, capacity planning

📋 listUsage Analysis (if available):
  ✅ Should provide: User, creator, project names
  ✅ Should include: CPU, GPU allocations, duration, status
  ✅ Good for: Detailed usage attribution and billing
  ✅ Better for: User accountability and project tracking
  
🎯 Ideal for: Monthly billing reports, user activity analysis

💡 RECOMMENDATION:
""")

if found_method:
    print(f"""
✅ Use BOTH methods for comprehensive reporting:
  - getTimeSeries: For resource utilization trends
  - {found_method}: For detailed usage attribution
  
🚀 Enhanced Monthly Report could include:
  1. Resource usage trends (from getTimeSeries)
  2. User activity breakdown (from {found_method})
  3. Project-level consumption (from {found_method})
  4. Top consumers and workloads (from {found_method})
  5. Duration and efficiency metrics (from {found_method})
""")
else:
    print("""
⚠️  Only getTimeSeries found - Limited reporting capability
  - Current reports show resource trends but no user context
  - Consider exploring CML documentation for additional APIs
  - May need to combine with other data sources for complete picture
  
🔍 Next steps:
  1. Check CML API documentation for usage/billing endpoints
  2. Look for job/session listing APIs that include resource info
  3. Consider workload-level APIs for user attribution
""")

print("\n" + "="*60)
print("4. NEXT STEPS")
print("="*60)

if found_method:
    print(f"""
1. Explore {found_method} method parameters and response structure
2. Create enhanced usage report combining both methods
3. Add user/project filtering and attribution
4. Include workload details (duration, status, efficiency)
5. Generate comprehensive monthly billing reports
""")
else:
    print("""
1. Research CML API documentation for additional usage methods
2. Check for job/session listing APIs with resource information
3. Look for workload or billing-specific endpoints
4. Consider combining multiple API calls for complete picture
5. Validate against actual CML environment capabilities
""")

print("\n=== Investigation Complete ===")

=== CML API Methods Comparison ===
Comparing getTimeSeries vs listUsage methods

1. ANALYZING getTimeSeries METHOD
✅ get_time_series method found
Signature: get_time_series(**kwargs)
Documentation preview: Return the time series data for the requested resource or property.  

        This method makes a synchronous HTTP request by default. To make an
        asynchronous HTTP request, please pass async_req=True
        >>> thread = api.get_time_series(async_req=True)
        >>> result = thread.get()
...

--- Testing getTimeSeries Response Structure ---
✅ getTimeSeries successful!
Response structure:
  - series_type: cpu
  - result keys: ['values']
  - Data points: 13
  - Sample data point: {'time_stamp': '1753968697667', 'count': '34'}
  - Available fields in data points: ['time_stamp', 'count']

📊 getTimeSeries provides:
  ✅ Time series data (timestamps + counts)
  ✅ Resource type (cpu/memory/gpu)
  ❌ NO user information
  ❌ NO workload details
  ❌ NO project information
  ❌ NO job n